In [8]:
import cv2
import numpy as np
import os
from glob import glob

# ==============================
# CONFIG
# ==============================
INPUT_DIR = r"C:\Users\3kp05\Desktop\Neural_Biopsy-main\new semester\all fascicles\set2_Image_01_20x_bf_05_all"
OUTPUT_DIR = r"C:\Users\3kp05\Downloads\blur_maps_set2_Image_01_20x_bf_05_all"
WINDOW_SIZES = [3, 5, 7]

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==============================
# Function: local Laplacian variance
# ==============================
def local_laplacian_variance(lap_img, k):
    mean = cv2.blur(lap_img, (k, k))
    mean_sq = cv2.blur(lap_img ** 2, (k, k))
    var = mean_sq - mean ** 2
    var = cv2.normalize(var, None, 0, 255, cv2.NORM_MINMAX)
    return var.astype(np.uint8)

# ==============================
# Process all PNG images
# ==============================
png_files = glob(os.path.join(INPUT_DIR, "*.png"))

if len(png_files) == 0:
    raise RuntimeError("No PNG images found in input directory!")

for img_path in png_files:
    filename = os.path.splitext(os.path.basename(img_path))[0]
    print(f"Processing: {filename}.png")

    # Read image
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Could not read {img_path}, skipping.")
        continue

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Output subfolder per image
    img_out_dir = os.path.join(OUTPUT_DIR, filename)
    os.makedirs(img_out_dir, exist_ok=True)

    # --------------------------
    # Save Gaussian low-pass outputs
    # --------------------------
    # for k in WINDOW_SIZES:
    #     gaussian = cv2.GaussianBlur(gray, (k, k), 0)
    #     gauss_path = os.path.join(img_out_dir, f"gaussian_{k}x{k}.png")
    #     cv2.imwrite(gauss_path, gaussian)

    # --------------------------
    # Denoising for Laplacian
    # --------------------------
    gray_denoised = cv2.GaussianBlur(gray, (3, 3), 0)

    # Laplacian
    lap = cv2.Laplacian(gray_denoised, cv2.CV_64F)
    lap = np.abs(lap)

    # --------------------------
    # Blur maps (multi-scale)
    # --------------------------
    for k in WINDOW_SIZES:
        blur_map = local_laplacian_variance(lap, k)
        out_path = os.path.join(img_out_dir, f"blur_map_{k}x{k}.png")
        cv2.imwrite(out_path, blur_map)

print("✅ Done processing all PNG images.")


Processing: 1.png
Processing: 2.png
Processing: 3.png
Processing: 4.png
Processing: 5.png
✅ Done processing all PNG images.
